In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np

In [ ]:
## 네 페이지당 최상위 기업 하나씩 뽑기

a = list(range(0,680,4))
len(a)

com_name_li = []
pos_li = []
for i in a:
    url = f'https://www.jobplanet.co.kr/companies?sort_by=review_avg_cache&page={i}'
    data = requests.get(url)
    soup = BeautifulSoup(data.content,'html.parser')
    a_tags = soup.find_all("a",{'class':"us_stxt_1"})
    names = soup.select('.us_titb_l3 a')
    pos_li.append(a_tags[0]["href"])
    com_name_li.append(names[0].text)
    
    print(a_tags[0]["href"],names[0].text)

In [ ]:
usr = "minyouil@o.cnu.ac.kr"
pwd = "az0319sx"

In [ ]:
## 네 페이지마다 하나씩 총 170개 회사의 리뷰 마지막 페이지 수 뽑기

driver = webdriver.Chrome("C:/Users/admin/chromedriver.exe")
driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")

time.sleep(2) ## 코드실행 2초간 정지

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)  ## send_keys -> 키보드 조작,입력 해주는 함수
login_pwd = driver.find_element_by_css_selector("input#user_password")
login_pwd.send_keys(pwd)
login_id.send_keys(Keys.RETURN)
time.sleep(2)

# driver.find_element_by_xpath("/html/body/div[1]/div[1]/header/nav/ul/li[1]/a").click()
# time.sleep(3)

# driver.find_element_by_xpath('//*[@id="top5Companies"]/section[6]/div/p/a').click()
# time.sleep(3)
name_pg = []
pages = []
for i in pos_li:
    url = f'https://www.jobplanet.co.kr{i}'
    data = driver.get(url)
    try:
        driver.find_element_by_css_selector("button.btn_close_x_ty1 ").click()
    except:
        pass
    soup = BeautifulSoup(driver.page_source,'html.parser')
    datas = soup.find_all("a",{'class':"btn_pglast"})
    names = driver.find_element_by_css_selector('body > div.body_wrap > div.cmp_hd > div.new_top_bnr > div > div.top_bnr_wrap > div > div > div.company_info_sec > div.company_info_box > div.company_name > h1 > a')
    pages.append(datas[0]["href"][-3:])
    name_pg.append(names.text)
    print(datas[0]["href"][-3:], names.text)
print(pages)
driver.close()

In [ ]:
## 'e','=' 삭제
pages = [page.strip('e=') for page in pages]

In [ ]:
## 회사별 리뷰(장점, 단점, 별점) 크롤링

driver = webdriver.Chrome("C:/Users/admin/chromedriver.exe")
driver.get("https://www.jobplanet.co.kr/users/sign_in?_nav=gb")

time.sleep(2) ## 코드실행 2초간 정지

login_id = driver.find_element_by_css_selector("input#user_email")
login_id.send_keys(usr)  ## send_keys -> 키보드 조작,입력 해주는 함수
login_pwd = driver.find_element_by_css_selector("input#user_password")
login_pwd.send_keys(pwd)
login_id.send_keys(Keys.RETURN)
time.sleep(2)


li_stars = []
li_merit = []
li_disad = []

for com,page in zip(pos_li,pages):
    for i in range(1,int(page)+1):
        url = f'https://www.jobplanet.co.kr{com}page={i}'
        data = driver.get(url)
        try:
            driver.find_element_by_css_selector("button.btn_close_x_ty1 ").click()
        except:
            pass
    
        stars = driver.find_elements_by_css_selector("div.star_score") 
        for j in stars:
            a = j.get_attribute('style')
            if a[7:9] == '20':
                li_stars.append("1점")
            elif a[7:9] == '40':
                li_stars.append("2점")
            elif a[7:9] == '60':
                li_stars.append("3점")
            elif a[7:9] == '80':
                li_stars.append("4점")
            else:
                li_stars.append("5점")
        
        li_reviews = []
        review = driver.find_elements_by_css_selector("dd.df1")
        
        count = int(len(review)/3)
        
        for j in review:
            li_reviews.append(j.text)

        for j in range(count):            
            a = li_reviews[3*j]
            li_merit.append(a)
        
            b = li_reviews[3*j+1]
            li_disad.append(b)

        try:
            driver.find_element_by_css_selector("a.btn_pgnext").click()
            time.sleep(2)
        except:
            pass

total_data = pd.DataFrame()
total_data['별점'] = pd.Series(li_stars)
total_data['장점'] = pd.Series(li_merit)
total_data['단점'] = pd.Series(li_disad)

total_data.to_csv("total_data.csv",index=False)

driver.close()